# Pointwise Methods

In this notebook, we compare pointwise methods and apply search methods to find the best performing one. First, we load the required dependencies and the data.

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd
from category_encoders.binary import BinaryEncoder
from category_encoders.one_hot import OneHotEncoder
from category_encoders.ordinal import OrdinalEncoder
from category_encoders.target_encoder import TargetEncoder

from src import configuration as config
from src.pipeline.pipeline_factory import PipelineFactory, ModelType, EvaluationType

In [ ]:
train_df = config.load_traindata_for_pointwise()
pipelineFactory = PipelineFactory()

## Pointwise Regression

In [ ]:
pipeline = pipelineFactory.create_pipeline(
    train_df,
    ModelType.POINTWISE_REGRESSION_NO_SEARCH,
    verbose_level=1,
    evaluation=EvaluationType.CROSS_VALIDATION,
    n_folds=5,
    workers=1
)

pipeline.run()

In [ ]:
param_grid = {
    "encoder_transformer__batch_size": [50, 500],
}

pipeline = pipelineFactory.create_pipeline(
    train_df,
    ModelType.POINTWISE_REGRESSION_NO_SEARCH,
    verbose_level=1,
    evaluation=EvaluationType.GRID_SEARCH,
    param_grid=param_grid,
    n_folds=2,
    workers=1
)

pipeline.run()

## Pointwise Classification

In [ ]:
pipeline = pipelineFactory.create_pipeline(
    train_df,
    ModelType.POINTWISE_CLASSIFICATION_NO_SEARCH,
    verbose_level=1,
    n_folds=5,
    workers=1
)

pipeline.run()

In [ ]:
param_grid = {
    "encoder_transformer__batch_size": [50, 500],
}

pipeline = pipelineFactory.create_pipeline(
    train_df,
    ModelType.POINTWISE_CLASSIFICATION_NO_SEARCH,
    verbose_level=1,
    evaluation=EvaluationType.GRID_SEARCH,
    param_grid=param_grid,
    n_folds=2,
    workers=1
)

pipeline.run()

## Pointwise Ordinal Regression

In [ ]:
pipeline = pipelineFactory.create_pipeline(
    train_df,
    ModelType.POINTWISE_ORDINAL_REGRESSION_NO_SEARCH,
    verbose_level=1,
    n_folds=5,
    workers=1
)

pipeline.run()

In [ ]:
param_grid = {
    "encoder_transformer__batch_size": [50, 500],
}

pipeline = pipelineFactory.create_pipeline(
    train_df,
    ModelType.POINTWISE_ORDINAL_REGRESSION_NO_SEARCH,
    verbose_level=1,
    evaluation=EvaluationType.GRID_SEARCH,
    param_grid=param_grid,
    n_folds=2,
    workers=1
)

pipeline.run()

## Comparison